In [1]:
#for small images

In [2]:
#for big images

In [ ]:
import os
import cv2
import glob
import numpy as np
from PIL import Image

# 1. Lift the pixel limit for both OpenCV and PIL
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(2**33) # Set to ~8.5 billion pixels
Image.MAX_IMAGE_PIXELS = None # Disable PIL safety limit entirely

def to_rgb(img_path, out_path=None):
    out_path = out_path or img_path
    
    try:
        # Attempt to read with OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        
        # If OpenCV fails or returns None, force use of PIL
        if img is None:
            raise ValueError("OpenCV returned None")

    except Exception:
        # Fallback to PIL for massive or problematic files
        try:
            print(f"Using PIL fallback for: {os.path.basename(img_path)}")
            with Image.open(img_path) as pill_img:
                # Convert to RGB and then to a NumPy array
                img_rgb = pill_img.convert('RGB')
                img = np.array(img_rgb)
                # OpenCV uses BGR, PIL uses RGB, so we swap
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"FAILED to read {img_path}: {e}")
            return

    # Process channels to ensure 3-channel RGB
    try:
        if img.ndim == 2:  # Grayscale
            img = cv2.merge([img, img, img])
        elif img.ndim == 3:
            if img.shape[2] == 4:  # RGBA
                img = img[:, :, :3]
            elif img.shape[2] == 3:
                pass  # Already RGB
            else:
                print(f"Skip (unexpected channels {img.shape[2]}): {img_path}")
                return
        
        # Save the result
        cv2.imwrite(out_path, img)
        print(f"Successfully processed: {os.path.basename(img_path)}")

    except Exception as e:
        print(f"Error during processing/saving {img_path}: {e}")

# --- Configuration ---
roots = [
    r"C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\inputiMage",
]
exts = ("*.png", "*.jpg", "*.jpeg", "*.tif", "*.tiff")

print("Starting processing...")
for root in roots:
    for ext in exts:
        # Recursive search
        files = glob.glob(os.path.join(root, "**", ext), recursive=True)
        for p in files:
            to_rgb(p)

print("Done: All images have been checked and converted.")

In [23]:
# import torch, torchvision, ultralytics
# print("Torch:", torch.__version__)
# print("TorchVision:", torchvision.__version__)
# print("Ultralytics:", ultralytics.__version__)
# print("CUDA available:", torch.cuda.is_available())
# if torch.cuda.is_available():
#     print("GPU:", torch.cuda.get_device_name(0))


In [24]:
#counting for big images

In [44]:
import os, math, cv2, numpy as np
from ultralytics import YOLO
from PIL import Image

# 1. CRITICAL: Lift limits for BOTH OpenCV and PIL
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(2**33)
Image.MAX_IMAGE_PIXELS = None 

# ----------------------------
# Config
# ----------------------------
IMAGE_PATH = r"C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/inputiMage/Canada24116.png"
MODEL_PATH = r"C:/Users/bazrafka/Desktop/counting/runs/detect/train14/weights/best.pt"
OUT_IMG    = r"C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/outputRESULTS/stitched_result.jpg"

TILE_SIZE  = 640          
OVERLAP    = 320           
IMGSZ      = TILE_SIZE     
CONF_TH    = 0.01           
IOU_NMS    = 0.2           
DRAW_MASKS = True          

model = YOLO(MODEL_PATH)

# ----------------------------
# Utilities
# ----------------------------
def iter_tiles(w, h, tile, overlap):
    stride = max(1, tile - overlap)
    xs = list(range(0, max(1, w - tile + 1), stride))
    ys = list(range(0, max(1, h - tile + 1), stride))
    if xs and xs[-1] + tile < w: xs.append(w - tile)
    if ys and ys[-1] + tile < h: ys.append(h - tile)
    for y in ys:
        for x in xs:
            x1, y1 = int(max(0, x)), int(max(0, y))
            x2, y2 = int(min(w, x1 + tile)), int(min(h, y1 + tile))
            yield x1, y1, x2, y2

def box_iou_xyxy(a, b):
    xx1 = max(a[0], b[0]); yy1 = max(a[1], b[1])
    xx2 = min(a[2], b[2]); yy2 = min(a[3], b[3])
    iw = max(0.0, xx2 - xx1); ih = max(0.0, yy2 - yy1)
    inter = iw * ih
    ua = (a[2]-a[0])*(a[3]-a[1]) + (b[2]-b[0])*(b[3]-b[1]) - inter
    return inter / ua if ua > 0 else 0.0

def nms_global(dets, iou_thr=0.5):
    out = []
    dets_by_cls = {}
    for d in dets:
        dets_by_cls.setdefault(d["cls"], []).append(d)
    for c, items in dets_by_cls.items():
        items = sorted(items, key=lambda d: d["conf"], reverse=True)
        while items:
            best = items.pop(0)
            out.append(best)
            items = [d for d in items if box_iou_xyxy(best["xyxy"], d["xyxy"]) < iou_thr]
    return out

def draw_global(image_bgr, detections, draw_masks=True):
    out = image_bgr.copy()
    for d in detections:
        x1,y1,x2,y2 = map(int, d["xyxy"])
        cv2.rectangle(out, (x1,y1), (x2,y2), (0,255,0), 2)
        if draw_masks and d.get("polys"):
            for poly in d["polys"]:
                if poly is not None and len(poly) > 0:
                    pts = np.round(poly).astype(np.int32).reshape(-1,1,2)
                    cv2.polylines(out, [pts], isClosed=True, color=(255,0,0), thickness=1)
    return out

# ----------------------------
# Pipeline (Modified for Large Images)
# ----------------------------

print(f"Loading image: {IMAGE_PATH}")
try:
    # Use PIL to load the image instead of cv2.imread
    with Image.open(IMAGE_PATH) as pil_img:
        # Convert to RGB then to BGR (OpenCV format)
        img = cv2.cvtColor(np.array(pil_img.convert("RGB")), cv2.COLOR_RGB2BGR)
except Exception as e:
    raise RuntimeError(f"Failed to load massive image with PIL: {e}")

H, W = img.shape[:2]
print(f"Image Resolution: {W}x{H} ({ (W*H)/1e6 :.2f} Million Pixels)")

global_dets = []

for x1,y1,x2,y2 in iter_tiles(W, H, TILE_SIZE, OVERLAP):
    tile = img[y1:y2, x1:x2]
    res = model.predict(tile, imgsz=IMGSZ, conf=CONF_TH, verbose=False)
    if not res: continue
    r = res[0]

    if getattr(r, "boxes", None) is not None and len(r.boxes) > 0:
        masks = r.masks.xy if getattr(r, "masks", None) is not None else None

        for i in range(len(r.boxes)):
            cls  = int(r.boxes.cls[i].item())
            conf = float(r.boxes.conf[i].item())
            xyxy = r.boxes.xyxy[i].detach().cpu().numpy()
            g_xyxy = xyxy + np.array([x1, y1, x1, y1], dtype=np.float32)

            polys_glob = None
            if masks is not None:
                try:
                    polys_tile = masks[i]
                    polys_glob = [p + np.array([x1, y1], dtype=np.float32) for p in polys_tile]
                except: polys_glob = None

            global_dets.append({
                "cls": cls, "conf": conf, "xyxy": g_xyxy.astype(float), "polys": polys_glob
            })

global_dets = nms_global(global_dets, iou_thr=IOU_NMS)
vis = draw_global(img, global_dets, draw_masks=DRAW_MASKS)

# Ensure output directory exists
os.makedirs(os.path.dirname(OUT_IMG), exist_ok=True)
cv2.imwrite(OUT_IMG, vis)

print(f"Done. Kept {len(global_dets)} detections. Saved to {OUT_IMG}")

Loading image: C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/inputiMage/Canada24116.png
Image Resolution: 20723x18671 (386.92 Million Pixels)
Done. Kept 40267 detections. Saved to C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/outputRESULTS/stitched_result.jpg


In [45]:
# Export shapes (Shapefile / GDB)

In [46]:
# ============================
# Export shapes (Shapefile / GDB)
# ============================
import os
from pathlib import Path

OUTPUT_DIR = Path(r"C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/outputRESULTS")  # change if needed
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_SHP = OUTPUT_DIR / "detections_boxes.shp"               # Shapefile
OUT_GDB = OUTPUT_DIR / "detections.gdb"                     # File Geodatabase (optional if arcpy available)
FC_NAME = "boxes"                                           # feature class name inside GDB

# ---- Try optional GIS modules ----
try:
    import geopandas as gpd
    from shapely.geometry import Polygon
    _HAS_GPD = True
except Exception as e:
    _HAS_GPD = False
    print("[WARN] geopandas/shapely not available -> Shapefile export will be skipped.", e)

try:
    import rasterio
    _HAS_RIO = True
except Exception:
    _HAS_RIO = False

try:
    import arcpy
    _HAS_ARCPY = True
except Exception:
    _HAS_ARCPY = False

# ---- Helper: pixel -> map coords if image is georeferenced ----
def _get_geo_context(image_path):
    """Return (transform, crs) or (None, None) if not available."""
    if not _HAS_RIO:
        return None, None
    try:
        with rasterio.open(image_path) as src:
            # Some JPEGs may have no CRS/transform; handle that
            transform = src.transform if src.transform is not None else None
            crs = src.crs if src.crs is not None else None
            # Identify uninitialized transform (common for plain JPEGs)
            if transform is not None and (transform.a == 1 and transform.e == -1 and transform.c == 0 and transform.f == 0):
                # Heuristic: this often indicates pixel space with flipped y
                # Treat as not-georeferenced for safety
                return None, None
            return transform, crs
    except Exception:
        return None, None

def _px_to_map_box(x1, y1, x2, y2, transform):
    """Return polygon coords in map units using rasterio Affine transform."""
    # Corner mapping: (col,row) -> (x,y)
    # Note: y is row, x is col in image terms
    from affine import Affine
    assert isinstance(transform, Affine)
    # Four corners in pixel space
    pts_px = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
    # Convert each (col,row) to (X,Y)
    pts_map = [transform * (px, py) for (px, py) in pts_px]
    return pts_map

# ---- Build GeoDataFrame rows ----
def _build_records(detections, transform=None, class_names=None):
    """Return list of dicts with geometry + attributes."""
    recs = []
    for i, d in enumerate(detections, 1):
        x1, y1, x2, y2 = map(float, d["xyxy"])
        w = max(0.0, x2 - x1)
        h = max(0.0, y2 - y1)
        area_px = w * h
        cls_id = int(d["cls"])
        name = class_names.get(cls_id, str(cls_id)) if class_names else str(cls_id)
        conf = float(d["conf"])

        if transform is not None:
            try:
                ring = _px_to_map_box(x1, y1, x2, y2, transform)
            except Exception:
                # Fallback to pixel coords if transform fails
                ring = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
        else:
            ring = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]

        recs.append({
            "id": i,
            "cls": cls_id,
            "class_name": name,
            "conf": conf,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "width": w, "height": h, "area_px": area_px,
            "geometry": Polygon(ring) if _HAS_GPD else None
        })
    return recs

# ---- Prepare class name mapping from the model (if available) ----
CLASS_NAMES = None
try:
    # Ultralytics models expose names as a dict {id: "name"}
    CLASS_NAMES = model.model.names if hasattr(model, "model") else model.names
except Exception:
    CLASS_NAMES = None

# ---- Read geo context from input image ----
rio_transform, rio_crs = _get_geo_context(IMAGE_PATH)

# ---- Build records ----
records = _build_records(global_dets, transform=rio_transform, class_names=CLASS_NAMES)

# ---- Export Shapefile via GeoPandas ----
if _HAS_GPD:
    gdf = gpd.GeoDataFrame(records, geometry="geometry", crs=rio_crs)  # crs=None if not georef'd
    # If not georeferenced, you can leave CRS unset; shapefile will be created without .prj
    # If you prefer to tag pixel space, omit setting a fake CRS to avoid confusion.
    if OUT_SHP.exists():
        OUT_SHP.unlink()  # remove stale file so drivers don't complain about schema changes
    # Driver chosen by extension (.shp)
    gdf.to_file(OUT_SHP)
    print(f"[OK] Shapefile written: {OUT_SHP}")
else:
    print("[SKIP] Shapefile export skipped (install geopandas + shapely + fiona).")

# ---- Optional: Export to File Geodatabase via arcpy (if available) ----
if _HAS_ARCPY:
    try:
        # Create GDB if missing
        if not OUT_GDB.exists():
            arcpy.management.CreateFileGDB(str(OUTPUT_DIR), OUT_GDB.name)
        # Delete FC if exists
        fc_path = str(OUT_GDB / FC_NAME)
        if arcpy.Exists(fc_path):
            arcpy.management.Delete(fc_path)

        # Spatial reference
        if rio_crs:
            # Convert rasterio CRS to WKT and feed to arcpy
            sr = arcpy.SpatialReference()
            sr.loadFromString(rio_crs.to_wkt())
        else:
            sr = arcpy.SpatialReference(0)  # Unknown

        # Create feature class (POLYGON)
        arcpy.management.CreateFeatureclass(str(OUT_GDB), FC_NAME, "POLYGON", spatial_reference=sr)

        # Add fields (keep names <= 10 chars for shapefile, but GDB is flexible; still keep tidy)
        fld_specs = [
            ("id", "LONG"),
            ("cls", "LONG"),
            ("cls_name", "TEXT", 64),
            ("conf", "DOUBLE"),
            ("x1", "DOUBLE"), ("y1", "DOUBLE"), ("x2", "DOUBLE"), ("y2", "DOUBLE"),
            ("width", "DOUBLE"), ("height", "DOUBLE"), ("area_px", "DOUBLE"),
        ]
        for name, ftype, *rest in fld_specs:
            length = rest[0] if rest else None
            if ftype == "TEXT" and length:
                arcpy.management.AddField(fc_path, name, ftype, field_length=length)
            else:
                arcpy.management.AddField(fc_path, name, ftype)

        # Insert rows
        with arcpy.da.InsertCursor(fc_path,
                                   ["SHAPE@",
                                    "id","cls","cls_name","conf",
                                    "x1","y1","x2","y2","width","height","area_px"]) as cur:
            for rec in records:
                # Build polygon geometry
                ring = rec["geometry"].exterior.coords[:] if rec.get("geometry") is not None else [
                    (rec["x1"], rec["y1"]),
                    (rec["x2"], rec["y1"]),
                    (rec["x2"], rec["y2"]),
                    (rec["x1"], rec["y2"]),
                    (rec["x1"], rec["y2"])  # close ring
                ]
                array = arcpy.Array([arcpy.Point(x, y) for (x, y) in ring])
                poly = arcpy.Polygon(array, sr)
                cur.insertRow([poly,
                               rec["id"], rec["cls"], rec["class_name"], rec["conf"],
                               rec["x1"], rec["y1"], rec["x2"], rec["y2"],
                               rec["width"], rec["height"], rec["area_px"]])
        print(f"[OK] FileGDB feature class written: {fc_path}")
    except Exception as e:
        print("[WARN] GDB export failed:", e)
else:
    print("[SKIP] FileGDB export skipped (arcpy not available).")

print(f"Tiles processed. Global detections kept: {len(global_dets)}")
print(f"Saved visualization: {OUT_IMG}")


[OK] Shapefile written: C:\Users\bazrafka\Desktop\counting\DiscussionPaperData\outputRESULTS\detections_boxes.shp
[SKIP] FileGDB export skipped (arcpy not available).
Tiles processed. Global detections kept: 40267
Saved visualization: C:/Users/bazrafka/Desktop/counting/DiscussionPaperData/outputRESULTS/stitched_result.jpg
